# HV Calibration

This notebook is a first attempt at performing calibration based on HV data. The calibration will be based on Matt's work [here (login required)](https://zzz.physics.umn.edu/!cdms/cdms/k100/run_summary/run_76#calibration_with_baseline_correction), rather than the previous, incorrect ladder calibration.

In [1]:
#Imports
import uproot
import glob
import math
import numpy as np
import pandas as pd
import datetime as dt
import lmfit as lf
import matplotlib.pyplot as plt
from matplotlib import cm
#%matplotlib inline

In [5]:
datapath = "RQData/"
ser = ["07220822_1828","07220826_1219","07220826_1536","07220826_2007","07220827_1153","07220830_1724"]#["07220702_0911"]
lab = ["0V","-4V","-21V","-65V","-84V","-84V"] #["data"]

nser = len(ser); e=[]; z=[];
for s in range(nser):
    e_chain = pd.DataFrame(); z_chain = pd.DataFrame();
    for file in glob.glob(datapath+ser[s]+"/umn*root"):
        #print(file)
        if e_chain.empty: #For first entry, generate columns
            e_chain = pd.concat([e_chain,uproot.open(file)["rqDir/eventTree"].pandas.df()],axis=1)
            z_chain = pd.concat([z_chain,uproot.open(file)["rqDir/zip1"].pandas.df()],axis=1)
        else:
            e_chain = pd.concat([e_chain,uproot.open(file)["rqDir/eventTree"].pandas.df()],ignore_index=True)
            z_chain = pd.concat([z_chain,uproot.open(file)["rqDir/zip1"].pandas.df()],ignore_index=True)
    e.append(e_chain); z.append(z_chain)
    
for i,x in enumerate(z):
    z[i] = pd.concat([x,e[i]],axis=1)

#print(list(z[0])) #really just verifying that I imported something

#extra simplifying code if we have only one series
#z = z[0]
#e = e[0]

#print(z)

[Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: []]


In [3]:
for x in z:
    x['crand'] = np.array([(y==1) for y in x['EventCategory']])
    x['PTwid'] = (x['PTWKf40'] - x['PTWKr40'])*1e6
    x['PTdbs'] = (x['PAbspost'] - x['PAbs'] + x['PBbspost'] - x['PBbs'] 
        + x['PCbspost'] - x['PCbs'] + x['PDbspost'] - x['PDbs'] + x['PEbspost'] - x['PEbs']
        + x['PFbspost'] - x['PFbs'])
    x['PTINTall_PTbscorr'] = x['PTINTall'] - ((x['PTdbs']<0)*0.5*x['PTdbs']*4096/x['PFnorm'])
    x['pt_keV'] = 7.738820e+07*x['PTOFamps']+1.653756e+13*x['PTOFamps']**2
    x['pt_keV_bscorr'] = x['pt_keV']/(1.-0.232*(x['PFbs']-3000.)/150) #baseline correction
    x['cam'] = x['PFOFamps']/x['PTOFamps']>0.21 #Americium cut
    x['xdel'] = 1e6*(x['PEWKr20'] - 0.5*(x['PCWKr20']+x['PDWKr20']))-1
    x['ydel'] = 1e6*(0.866*(x['PDWKr20'] - x['PCWKr20']))+9
    x['phidel'] = [180/np.pi*math.atan2(x['ydel'].array[j],x['xdel'].array[j]) for j in range(len(x['ydel'].array))]
    x['cphi1'] = x['phidel']>5 and x['phidel'] < 20

KeyError: 'EventCategory'

To get the aliases right, trying to reproduce: https://zzz.physics.umn.edu/!cdms/cdms/k100/run_summary/run_76#calibration_with_baseline_correction

In [ ]:
fig = plt.figure()
h = plt.hist2d(z[4]["PFbs"][~z[4]["crand"]][z[4]["cphi1"]][z[4]["cam"]]-3000,z[4]["pt_keV"][~z[4]["crand"]][z[4]["cphi1"]][z[4]["cam"]]
               ,bins=[np.linspace(-50,400,200),np.linspace(0,300,150)])
plt.ylabel("Phonon energy [keV]")
plt.xlabel("Baseline elevation") #Not true yet; just the goal. 
#Based on the calculation I think this might be what I have now? But,
#Based on the plot it's very clearly not it.
fig.colorbar(h[3])
x = np.array([0,300])
plt.plot(x,-0.232*x+150,color='r',linestyle='--')
#plt.plot([0,300],[150,75])

comment = """
pointsx = [0,0.22e-6,0.29e-6,0.38e-6] #Measured outputs, not yet known.

plt.axhline(pointsx[1],color='red',linestyle='dotted')
plt.axhline(pointsx[2],color='red',linestyle='dotted')
plt.axhline(pointsx[3],color='red',linestyle='dotted')#"""

plt.show()

In [ ]:
fig = plt.figure()
h = plt.hist2d(z[4]["PFbs"][~crand[4]]-3000,z[4]["pt_keV_bscorr"][~crand[4]]
               ,bins=[np.linspace(-50,400,200),np.linspace(0,300,150)])
plt.ylabel("Baseline-corrected phonon energy [keV]")
plt.xlabel("Baseline elevation") #Not true yet; just the goal. 
#Based on the calculation I think this might be what I have now? But,
#Based on the plot it's very clearly not it.
fig.colorbar(h[3])

comment = """
pointsx = [0,0.22e-6,0.29e-6,0.38e-6] #Measured outputs, not yet known.

plt.axhline(pointsx[1],color='red',linestyle='dotted')
plt.axhline(pointsx[2],color='red',linestyle='dotted')
plt.axhline(pointsx[3],color='red',linestyle='dotted')#"""

plt.show()